# Log for project in Computer Science

By Anna and Eric

## Day 1, 4/4 - 2019

Today the goal is to understand the project and be prepared for mondays meeting.

Step 

     1) Download UniProt and check out the dataset

     2) Create the dictionary
     
     3) Get tool from Marcus
     
     4) Train a tagger with machine learning
     
So we would like to finish step 1 and start reseach on 2.

---
Today we sat from 13-16. We started by creating a private git repo with this file in it. Then we downloaded the uniprot dataset in xml form. We proceded to try and parse it with BeautifulSoup. The file was too big for that method.

After that we tried with ElementTree instead which can go through each element one by one. This worked and we eventually got a list of proteins (over 200 000).

We got stuck at which dictionary we want to create. Do we want one that's protein -> gene1 gene2 gene3 and thus also mine genes from the xml file. Or is the dictionary something else? For next time this needs to be cleared up in order to move on.

In [ ]:
#Goes through the file uniprot_sprot.xml and saves the names of the proteins
#in protein_names.txt
from xml.etree import ElementTree as ET
filename='uniprot_sprot.xml'
parser = ET.iterparse(filename)
length = 28 #length of header in xml tags
proteins = set()
for event, element in parser:
    tag = element.tag[length:]
    if 'fullName' in tag or 'shortName' in tag:
        proteins.add(element.text)
    element.clear()
f = open('protein_names.txt', 'w')
f.write('\n'.join(proteins))
f.close()


## Day 2, 8/4 - 2019

Today we had our second meeting with Pierre and Sonja. We got clarification about the biology bit of the project but also some tips going forward. Now we have a Slack and are to get an invite to the Github repo. Also, an invite to Endnote.

TODO: Invite Sonja, Pierre to Overleaf document

Tips:

    1) Search PubMed for 'biomedical text-mining' and 'bio NLP' (Jensen)
    
    2) Get annotation from Hannes

So this week we wish to get the infrastructure up and running.

---
We started by looking at Marcus email and getting his program to work. After a lot of problems with installation of the various packages and programs we finally got it to work, almost 2h later.

Question: What is the file 'disease_index.fst' and how is it created? Should we create this?

---
We now have the basic infrastructure up and running. We have done some tests on running it with our dataset of proteins from UniProt. Also we started looking at BioCreative's datasets, but we'll leave that for next time. 


### Instructions to start JVM (need Marcus repo):

    1) cd [katalog]/mention-index-py4j

    2) mvn package

    3) cd target

    4) java -jar mentions-index-py4j-1.0-SNAPSHOT.jar

In [ ]:
# Install a conda package in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install docria
#!{sys.executable} -m pip install py4j
from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path
from docria.printout import options
set_large_screen()
def connect_jvm(port):
    from py4j.java_gateway import GatewayParameters, JavaGateway
    gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
    gateway = JavaGateway(gateway_parameters=gateway_parameters)
    app = gateway.entry_point
    return gateway, gateway.jvm, app
gateway, jvm, app = connect_jvm(6006)
def get_java_file(path):
    # Here we do the py4j equivalent for new java.io.File(path)
    return jvm.java.io.File(os.path.abspath(path))

In [ ]:
app.buildIndex(get_java_file("protein_names.txt"), get_java_file("disease_index.fst"))
indx = app.loadIndex(get_java_file("disease_index.fst"))
hits = indx.search("Multiple system atrophy (MSA) is a progressive, neurodegenerative disease characterized by parkinsonism, resistance to dopamine therapy, ataxia, autonomic dysfunction, and pathological accumulation of α-synuclein (α-syn) in oligodendrocytes. Neurosin (kallikrein-6) is a serine")
for hit in hits:
    print(hit.start, hit.end, " ".join([term.raw for term in hit.terms]))

## Day 3, 9/4-2019
We have had some questions about the purpose and think we have finally figured it out. To identify the entities 'protein' and 'diseases' like cell death. To start with we will identify proteins by checking a pubmed article and using our uniprot list.



In [ ]:
#Goes through the file pubmed19n0651.xml and saves the abstract text as one textfile
from xml.etree import ElementTree as ET
filename='pubmed19n0651.xml'
out_name = 'abstract_text19n0651.txt'
parser = ET.iterparse(filename)
f=open(out_name, "a+", encoding="utf-8")
for event, element in parser:
    if element.tag == 'AbstractText':
        if element.text:
            f.write(element.text)
    element.clear()
f.close()

In [ ]:
from docria import Document, MsgpackCodec, DocumentIO, set_large_screen, T
import os.path
from docria.printout import options
set_large_screen()
def connect_jvm(port):
    from py4j.java_gateway import GatewayParameters, JavaGateway
    gateway_parameters=GatewayParameters(port=port, auto_convert=True, auto_field=True)
    gateway = JavaGateway(gateway_parameters=gateway_parameters)
    app = gateway.entry_point
    return gateway, gateway.jvm, app
gateway, jvm, app = connect_jvm(6006)
def get_java_file(path):
    return jvm.java.io.File(os.path.abspath(path))

In [ ]:
#Prints all occurences of proteins in the abstract file.
app.buildIndex(get_java_file("protein_names.txt"), get_java_file("protein_name_index.fst"))
indx = app.loadIndex(get_java_file("protein_name_index.fst"))
data = ''
with open('abstract_text19n0651.txt', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
sp = data.split('.')
for s in sp:
    hits = indx.search(s)
    for hit in hits:
        print(hit.start, hit.end, " ".join([term.raw for term in hit.terms]))

In [ ]:
#Prints all occurences of cell deaths in the abstract file.
app.buildIndex(get_java_file("cell_death_names.txt"), get_java_file("cell_name_index.fst"))
indx = app.loadIndex(get_java_file("cell_name_index.fst"))
data = ''
with open('abstract_text19n0651.txt', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
sp = data.split('.')
for s in sp:
    hits = indx.search(s)
    for hit in hits:
        print(hit.start, hit.end, " ".join([term.raw for term in hit.terms]))

Next we tried to use the model that https://aclweb.org/anthology/W16-5104 implemented at github: https://github.com/withtwist/medical-ner. After transpiling the code to Python 2, changing the encoding of the data etc. we still got compilation errors. It seemed too big of a job so we scraped it. Since we have been able to identify proteins from the abstract and where they are located, we are now interested in finding a way to create the tracer with machine learning. We found the article *Unsupervised biomedical named entity recognition: Experiments with clinical and biological texts. 


Note: We noticed that the indx.search is case sensitive, which needs to be fixed. (e.g. toLowerCase or similar). Check w Marcus!!

Conclusion of todays work: We have fixed the dictionaries and spent some time making sure the xml reader for uniprot included the edgecases and included 'shortName'. We also started tinkering with neural networks by downloading existing repos but we could not make it work. We started reading a couple of papers to get inspiration for our tagger. 

For next time, we will look deeper into the neural networks and use the tips Marcus sent us.

## Day 4, 14/4-2019
We prepared for tomorrows meeting by looking at Marcus tips that he sent us. Also we started looking at more journals and papers and found a really good one: [A Novel Approach for Protein-Named Entity Recognition and Protein-Protein Interaction Extraction](https://www.hindawi.com/journals/mpe/2015/942435/). They describe step-by-step a method with SVM to extract the entities. If they agree on tomorrows meeting we will follow this guide next week.

## Day 5, 15/4-2019
Today after the meeting we have a clearer understanding how to move forward. The process follows:
    1. Add list of corpus with summarization of each content and size to github
    2. Fix java to ignore case
    3. Complete a working model with the dictionary model
        i. Fix reader and use Element tree instead. Also read in 'pmid' and change output to 'pmid \t abstract_text \n'.
        ii. Match output to corpus annotation
    4. Create evaluator for dictionary model
        i. Use [genetag](https://www.ncbi.nlm.nih.gov/pubmed/15960837) for evaluation or chemner? or both?
        ii. Compare with e.g recall
    5. Add more to dictionary and run evaluator to see if it improves. If ambigueties use dominant write.
    
After this is completed we will start on the machine learning model.
    1. Use POS? Pre-processing?
    2. Sentence -> Tokens -> Tagger -> Phrases -> ML (BioLSTM and Bioascembedding(spelling?))
    3. Check out page [nlpprogress](http://nlpprogress.com/english/named_entity_recognition.html) for NER datasets
   

Möte med Sonja: https://biocreative.bioinformatics.udel.edu/resources/corpora/biocreative-iii-corpus/



In [ ]:
indx = app.loadIndex(get_java_file("protein_name_index.fst"))

data = ''
with open('genetag.db', 'r', encoding="utf-8", errors='ignore') as file:
    data = file.read()
sp = data.split('>>')
dictionary = {}
for s in sp:
    lines = s.split('\n')
    #If it is an excerpt
    if lines[0] == 'EXCERPT':
        id = ''
        text = ''
        for line in lines[1:]:
            if line.startswith('EXCERPT_ID'):
                id = line.split(': ')[1]
            elif line.startswith('TEXT'):
                text = ''.join(line.split(': ')[1:])
        dictionary[id] = text
print(dictionary)
                

---
We tried using t

In [42]:
indx = app.loadIndex(get_java_file("protein_name_index.fst"))

d = dictionary
out = []
for key in d:
    value = d[key]
    hits = indx.search(value)    
    for hit in hits:
        # term.raw for term in hit.terms prints lists of protein names. Each list is protein name
        out.append('|'.join([key, str(hit.start) + ' ' + str(hit.end), " ".join([term.raw for term in hit.terms])]))
file = open('genetag.out', 'w+', encoding="utf-8", errors='ignore')
file.write('\n'.join(out))
file.close()
    